<img src="escudo_utfsm.gif" style="float:right;height:100px">
<img src="IsotipoDIisocolor.png" style="float:left;height:100px">
<center>
    <h1> ILI285 - Computación Científica I / INF285 - Computación Científica</h1>
    <h1> Tarea 4: PageRank y GMRes </h1>
    <h2> Ignacio Cisternas Núñez -- ROL: 201573544-1</h2>
    <h4> ignacio.cisternasn@sansano.usm.cl </h4>
</center>



In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random
import scipy as sp
from time import time
from ipywidgets import interact, IntSlider
import networkx as nx
from scipy.sparse.linalg import gmres
from scipy import sparse

## PageRank

Para poder trabajar con el algoritmo, es necesario considerar inicialmente una matriz de adyacencia $A \in \mathbb{R}^{n \times n}$, con $n$ la cantidad de páginas web. Las entradas $a_{ij}$ de esta matriz tienen el valor 1 si la página $i$ tiene un enlace a la página $j$ y 0 en caso contrario. Notar que no necesariamente la matriz $A$ es simétrica, lo que denota que dos páginas web distintas podrían no enlazarse mutuamente. Además, considere que una misma página no se enlazará consigo misma, por lo que la matriz tendrá cero en su diagonal principal. 

Adicionalmente, podrían darse casos de que existan páginas que solo tienen links hacia ellas, pero no tienen links hacia otras páginas. En una representación de la matriz de adyacencia como grafo, se le conoce a estas páginas como nodos _sumideros_. Una consecuencia de estos casos podía ser que usuarios que llegan a esas páginas quedan retenidos porque no existen links a los cuales seguir navegando. Para evitar esta situación, se agregará la perturbación _rank-one_ a la matriz de adyacencia $A$.

$$
    \tilde{A} = A + \mathbf{a}\cdot\mathbf{1}^T,
$$

donde $\mathbf{a} \in \mathbb{R}^{n}$ es un vector con un $1$ en la componente que corresponde a los nodos sumideros  y $0$ en los otras componentes, el vector $\mathbf{1}$ corresponde al vector de unos en $\mathbb{R}^{n}$ y $^T$ es el operador transpuesta.

#### Dataset
Considere los datasets de los archivos `adjacency1.dat`, `adjacency2.dat`, `adjacency3.dat` y `adjacency4.dat`, que representan matrices de adyacencia tales que:
- Adjacency1 es una matriz de adyacencia de 100 páginas y alrededor de un 20% de elementos no nulos.
- Adjacency2 es una matriz de adyacencia de 100 páginas y alrededor de un 50% de elementos no nulos.
- Adjacency3 es una matriz de adyacencia de 100 páginas y alrededor de un 80% de elementos no nulos.
- Adjacency4 es una matriz de adyacencia de 1000 páginas y alrededor de un 5% de elementos no nulos.

Cada fila $i$ de un archivo dataset corresponde a una página de índice $i$, y todos los valores separados por espacios en dicha fila representan los índices de las páginas $j$ a las cuales $i$ apunta. En otras palabras, un archivo dataset registra los vértices del grafo de adyacencia.

Considere la siguiente función,`read_adjacency_matrix` , que obtiene la matriz de adyacencia a partir de los archivos anteriores.

In [2]:
def read_adjacency_matrix(file_path):
    adjacency_list = []
    with open(file_path, 'r') as f:
        for line in f:
            adjacency_list.append(np.array(list(map(int, line.split()))))
    n = len(adjacency_list)
    A = np.zeros((n,n))
    for i in range(n):
        A[i, adjacency_list[i]] = 1
    return A
A = read_adjacency_matrix("adjacency1.dat")
B = read_adjacency_matrix("adjacency2.dat")
C = read_adjacency_matrix("adjacency3.dat")
D = read_adjacency_matrix("adjacency4.dat")

## Sección 1 : Comparación de soluciones con GMRes y PALU

En esta sección se compararán las soluciones de PageRank obtenidas por medio de GMRes y PALU. Para esto, se considerarán los datasets de los archivos `adjacency1.dat`, `adjacency2.dat`, `adjacency3.dat` y `adjacency4.dat`, variaciones en el _damping factor_ $\alpha$ y el número de iteraciones $k$ de GMRes.

**1.** Construya el sistema lineal necesario para encontrar PageRank. Para ello desarrolle la función `build_linear_system`, que recibe una matriz de adyacencia $A$ y un _damping factor_ $\alpha$.

In [3]:
'''
Input:
A - (n x n matrix) adjacency matrix
alpha - (float) damping factor, takes values from 0 to 1
Output: 
A_hat - (n x n matrix) matrix of linear system
b_hat - (n vector) right hand side vector of linear system
'''

def build_linear_system(A, alpha):
    largo = len(A)
    MatP = np.zeros((largo,largo))
    MatP = A
    vector1 = 0
    for x in MatP:
        suma = sum(x)
        vector2 = 0
        for j in x:
            if j == 1:
                MatP[vector1][vector2] = (1/suma)
            vector2 += 1
        vector1 += 1
    MatI = np.zeros((largo,largo))
    np.fill_diagonal(MatI,1)
    MatV = np.full((largo, 1), 1/largo)
    hat2 = np.zeros((largo,largo))
    hat2 = (1-alpha) * MatV
    hat1 = np.zeros((largo,largo))
    hat1 = MatI - (alpha * np.transpose(MatP))               
    return hat1, hat2

**2.** Considere el error $e_k = \|\mathbf{x^{k}_{G}}-\mathbf{x_{P}}\|_2$ una métrica de error que compara $\mathbf{x_{P}}$, la solución de PageRank obtenida por PALU, con $\mathbf{x^{k}_{G}}$ la solución de PageRank obtenida con $k$ iteraciones de GMRes. Construya un gráfico que muestre $e_k$ versus $k$ y utilice un widget para seleccionar un dataset y variar el valor del _damping factor_ $\alpha$. ¿Qué puede decir de la información mostrada en el gráfico? ¿Cómo afecta $\alpha$ en los resultados obtenidos?

In [6]:
def matrizTriangular(A, b, upper=True):
    n = b.shape[0]
    x = np.zeros_like(b)
    if upper==True:
        #perform back-substitution
        x[-1] = (1./A[-1,-1]) * b[-1]
        for i in range(n-2, -1, -1):
            x[i] = (1./A[i,i]) * (b[i] - np.sum(A[i,i+1:] * x[i+1:]))
    else:
        #perform forward-substitution
        x[0] = (1./A[0,0]) * b[0]
        for i in range(1,n):
            x[i] = (1./A[i,i]) * (b[i] - np.sum(A[i,:i] * x[:i]))
    return x

def palu_decomp(A):
    N,_ = A.shape
    P = np.identity(N)
    L = np.zeros((N,N))
    U = np.copy(A)
    for column in range(N-1):    #columnas
        p_index = np.argmax(np.abs(U[column:,column]))   #pivote
        if p_index != 0:
            row_perm(P, column, column+p_index)
            row_perm(U, column, column+p_index)
            row_perm(L, column, column+p_index)
        for row in range(column+1,N):   #filas 
            L[row,column] = U[row,column]/U[column,column]
            U[column] -= L[row,column]*U[column]
    np.fill_diagonal(L,1)
    return P,L,U

def GMRes(A, b, x0=np.array([0.0]), m=10, flag_display=False, threshold=1e-12):
    todas=np.zeros((m,len(b)))
    n = len(b)
    if len(x0)==1:
        x0=np.zeros(n)
    r0 = b - np.dot(A, x0)
    nr0=np.linalg.norm(r0)
    out_res=np.array(nr0)
    Q = np.zeros((n,n))
    H = np.zeros((n,n))
    Q[:,0] = r0 / nr0
    flag_break=False
    matris=0
    for k in np.arange(np.min((m,n))):
        y = np.dot(A, Q[:,k])
        if flag_display:
            print('||y||=',np.linalg.norm(y))
        for j in np.arange(k+1):
            H[j][k] = np.dot(Q[:,j], y)
            if flag_display:
                print('H[',j,'][',k,']=',H[j][k])
            y = y - np.dot(H[j][k],Q[:,j])
            if flag_display:
                print('||y||=',np.linalg.norm(y))
        if k+1<n:
            H[k+1][k] = np.linalg.norm(y)
            if flag_display:
                print('H[',k+1,'][',k,']=',H[k+1][k])
            if (np.abs(H[k+1][k]) > 1e-16):
                Q[:,k+1] = y/H[k+1][k]
            else:
                flag_break=True
            e1 = np.zeros((k+1)+1)        
            e1[0]=1
            H_tilde=H[0:(k+1)+1,0:k+1]
        else:
            H_tilde=H[0:k+1,0:k+1]
        ck = np.linalg.lstsq(H_tilde, nr0*e1)[0] 
        if k+1<n:
            x = x0 + np.dot(Q[:,0:(k+1)], ck)
        else:
            x = x0 + np.dot(Q, ck)
        norm_small=np.linalg.norm(np.dot(H_tilde,ck)-nr0*e1)
        out_res = np.append(out_res,norm_small)
        if flag_display:
            norm_full=np.linalg.norm(b-np.dot(A,x))
        todas[matris]=x
        matris=matris+1
        if flag_break:
            if flag_display: 
                print('EXIT: flag_break=True')
            break
        if norm_small<threshold:
            if flag_display:
                print('EXIT: norm_small<threshold')
            break
    return todas,out_res


In [8]:

def error(data,alpha,k):
    e=[]
    posicion = [A,B,C,D]
    for i in range (1,5):
        if data == i:
            Q,W = build_linear_system(posicion[i-1],alpha)
    P,L,U = palu_decomp(Q)  # SOLVE PALU 
    W = np.dot(P,W)
    d = matrizTriangular(L, W, upper=False)
    x = matrizTriangular(U, d)
    aux =(1/sum(x))
    resultado = aux *x
    W = np.transpose(W)
    algo2, _ = GMRes(Q, W[0],m=k)
    
    for j in algo2:
        algo = np.zeros((len(j),1))
        aux = 0
        for i in j:
            algo[aux] = i
            aux += 1
        simple = algo - resultado
        err = np.linalg.norm(simple)
        e.append(err)
    lista = range(1,len(e) + 1)
    vx = list(lista)
    plt.scatter(vx, e)
    plt.show()
    
    return False
np.warnings.filterwarnings('ignore')
interact(error,data=IntSlider(min=1,max=4,step=1,value=2),k=IntSlider(min=1,max=100,step=1,value=7),alpha=(0.1,1.0,0.01))

interactive(children=(IntSlider(value=2, description='data', max=4, min=1), FloatSlider(value=0.55, descriptio…

<function __main__.error(data, alpha, k)>

    
    --- R) Al variar el valor de Alpha se observan cambios en los errores involucrados. Más especificamente, disminuir el valor de alpha implica disminuir el error obtenido.
    

## Sección 2 : Tiempo de Ejecución

En esta sección se compararán los tiempos de ejecución de GMRes y PALU necesarios para resolver los sistemas de ecuaciones de PageRank. Para esto, se considerarán los datasets de los archivos `adjacency1.dat`, `adjacency2.dat`, `adjacency3.dat` y `adjacency4.dat`, variaciones en el _damping factor_ $\alpha$ y el número de iteraciones $k$ de GMRes.

**1.** Analice efecto de variar _damping factor_ $\alpha$ para encontrar las 10 primeras páginas entregadas por PageRank. Para ello utilice la función `get_damping_ranking` definida a continuación. 

In [9]:
'''
Input:
A - (n x n matrix) adjacency matrix
alpha - (float) damping factor, takes values from 0 to 1
k - number of iterations of GMRes until return a solution, use only if method is 'GMRes'
method - string that indicates the method used to solve the linear system. Take values 'PALU' or 'GMRes'
Output: 
ranking - list with 10 pages of ranking sorted by largest probability
'''

def get_damping_ranking(A ,alpha ,k ,method='GMRes'):
    Ma,Vb=build_linear_system(A, alpha)
    P,L,U = palu_decomp(Ma)     #DESCOMPOCISiON DE PALU
    Vb = np.dot(P,Vb)
    triangular1 = matrizTriangular(L, Vb, upper=False)
    triangular2 = matrizTriangular(U, triangular1)
    aux = (1/sum(triangular2))
    Prim = aux*triangular2
    Vb = np.transpose(Vb) 
    if "GMRes"== method:
        A1, _ = GMRes(Ma, Vb[0],m=k)
        A=np.zeros((len(Ma),1))
        A=A1[len(A1)-1]
        primerasPag=[]   #primeras Paginas de Pagerank
        aux2 = A
        for i in range(1,11):
            pag = aux2.argsort()[-i]
            primerasPag.append(np.array((pag+1, aux2[pag])))
        ranking = primerasPag
    else:
        B=np.transpose(Prim)[0]
        primerasPag=[]    #primeras Paginas de Pagerank
        aux2 = Prim
        for i in range(1,11):
            pag = aux2.argsort()[-i]
            primerasPag.append(np.array((pag+1, aux2[pag])))  
        ranking=primerasPag   
    return ranking

get_damping_ranking(B,0.2,5)

[array([8.40000000e+01, 1.04441278e-02]),
 array([3.10000000e+01, 1.04126977e-02]),
 array([7.90000000e+01, 1.03869554e-02]),
 array([5.        , 0.01038598]),
 array([9.60000000e+01, 1.03609736e-02]),
 array([5.60000000e+01, 1.03258035e-02]),
 array([1.00000000e+02, 1.02885179e-02]),
 array([1.        , 0.01026802]),
 array([2.40000000e+01, 1.02603393e-02]),
 array([4.30000000e+01, 1.02433969e-02])]

**2.** Construya un gráfico que muestre el tiempo de ejecución para determinar el ranking versus el factor de amortiguamiento $\alpha$. En el mismo gráfico debe mostrar los dos métodos utilizados (GMRes y PALU). Además, utilice un widget que permita seleccionar uno de los cuatro datasets mencionados y el número $k$ de iteraciones de GMRes. ¿Qué puede decir respecto de los resultados obtenidos en cada método al variar el valor de $\alpha$?

In [10]:
def tiempo (data,k,method):
    ejey,ejex = [],[]
    for i in range (1,5):
        if data == i:
            mat = read_adjacency_matrix("adjacency"+str(i)+".dat")
    alpha = 0
    for i in range(1,20):        
        t0 = time()
        if method == 1:
            get_damping_ranking(mat, alpha, k, method='PALU')
        elif method == 2:
            get_damping_ranking(mat, alpha, k, method='GMRes')
        t1 = time()
        ejey.append(alpha)
        alpha += 0.05
        tTotal = t1 - t0
        ejex.append(tTotal)
    plt.scatter(ejex, ejey)
    plt.show()
    return 0

interact(tiempo,method=(1,2,1),data=IntSlider(min=1,max=4,step=1,value=1),k=IntSlider(min=1,max=100,step=1,value=13))

#method == 1 ----> PALU  | method == 2 ---->GMRes

interactive(children=(IntSlider(value=1, description='data', max=4, min=1), IntSlider(value=13, description='k…

<function __main__.tiempo(data, k, method)>

## Sección 3 : Análisis de iteraciones de GMRes

En esta sección debe analizar las soluciones obtenidas por GMRes en cada iteración, utilizando los datasets `web-NotreDame` y `web-Stanford`. Se recomienda modificar el código de GMRes de los Jupyter Notebook del curso, aunque no es obligatorio. **Importante:** Debido al tamaño de estos datasets, no se debe intentar cargar toda la matriz de adyacencia en memoria en formato denso

Considere la relación error $e_{k}$ versus iteración $k$, donde el error puede ser definido de la siguiente manera:

$$
e_{k} = \| \mathbf{x}_{k}-\mathbf{x}_{k-1} \|_2
$$

   Donde $\mathbf{x}_k$ es la solución de GMRes obtenida en la iteración $k$-ésima, con $k$ que **puede tomar valores** en el rango $[1, 2, \ldots, m]$. y $m$ el número de páginas del dataset. 
   
**1.** Utilice GMRes de manera conveniente para graficar el error $e_k$ versus $k$, utilizando un widget para variar el _damping factor_ $\alpha$ y seleccionar uno de los dos datasets requeridos. ¿Qué puede decir del error a medida que $k$ aumenta? ¿En qué afecta el valor de $\alpha$?

**Recomendación:** no intente cargar toda la matriz $\widehat{A}$ en memoria. En lugar de eso, considere que debido a que $\widehat{A}$ es _sparse_, la matriz $P$ también lo es y evite el cálculo de productos exteriores explícitamente. Se recomienda revisar el módulo `sparse` de `scipy` . Puede recurrir a modificaciones de GMRes para desarrollar esta pregunta. Utilice un valor máximo de $k$ razonable, pero no muy pequeño. No debe llegar necesariamente a $k = m$. Justifique su elección apropiadamente.

## Sección 4 (10 puntos): Conclusiones

A partir de lo desarrollado en esta tarea concluya acerca de la pertinencia de utilizar GMRes para encontrar PageRank. ¿Qué ventajas y desventajas tiene el uso de este método en este problema? Comente.

   Se puede visualizar en esta actividad lo ventajoso que resulta utilizar el metodo del residuo minimo cuadrados para matrices dispersas que involucran sistemas lineales de gran tamaño como lo es la matriz de google. Que involucra una gran cantidad de información, Se observan errorres menores que en metodo PALU previamente estudiado. Lo que mejora la calidad de los datos. Esto pues como lo indica su nombre apunta a minimizar lo más posible el error de cada dato.

### Referencias

* https://github.com/tclaudioe/Scientific-Computing/blob/master/SC1/10_GMRes.ipynb

* https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.linalg.gmres.html

* Jupyter_Notebook_04_Metodos_Directos. (Material del curso)